In [154]:
from scipy.stats import spearmanr

import pandas as pd
import numpy as np
import os

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC 
from sklearn.ensemble import GradientBoostingClassifier

import lightgbm as lgbm
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
import string

import itertools

np.random.seed(1337)

In [ ]:
with open("/usr/share/dict/words") as wordfile:
    words = set(x.strip().lower() for x in wordfile.readlines())

In [ ]:
def spearmancorr(est,X,y):
    rho, pval = spearmanr(np.reshape(y, (-1, 1)), np.reshape(est.predict(X), (-1, 1)), axis=0)
    return rho

In [404]:
def process_data(path, exclude=[], train=True):
    def process_file(path):
        user = {'Alice': "A",'Bob': "B"}
        
        df = pd.read_json(path).set_index("dialogId")
        df['speaker'] = df["thread"].apply(lambda x: [user[msg['userId']] for msg in x])
        df['thread'] = df["thread"].apply(lambda x: [msg['text'] for msg in x], convert_dtype=False)
        df['thread_raw'] = df["thread"].apply(lambda x: " ".join(x))
        if train:
            df["qualA"] = df["evaluation"].apply(lambda x: sorted(x, key=lambda x: x['userId'])[0]['quality'])
            df["qualB"] = df["evaluation"].apply(lambda x: sorted(x, key=lambda x: x['userId'])[1]['quality'])
            df["botA"] = df["users"].apply(lambda x: sorted(x, key=lambda x: x['id'])[0]['userType'] == 'Bot')
            df["botB"] = df["users"].apply(lambda x: sorted(x, key=lambda x: x['id'])[1]['userType'] == 'Bot')
        df.drop(['users'], axis=1, inplace=True)
        if train:
            df.drop(['evaluation'], axis=1, inplace=True)
            
        return df
    
    def add_features(data):
        def preprocess(text, lower, punctuation):
            if lower:
                text = text.lower()
            if punctuation == "exclude":
                text = text.translate(str.maketrans({p: None for p in string.punctuation}))
            elif punctuation == "separate":
                text = text.translate(str.maketrans({p: " {} ".format(p) for p in string.punctuation}))

            return text

        punct_modes = ["exclude", "separate", "leave"]
        lower_modes = [True, False]
                
        for preproc_mode in itertools.product(lower_modes, punct_modes):            
            preproc = lambda text: preprocess(text, *preproc_mode)

            def get_speaker(speaker):
                return lambda row: [preproc(x) for x in np.array(row['thread'])[np.array(row['speaker']) == speaker]]

            data["thread_split_A_{}_{}".format(*preproc_mode)] = data.apply(get_speaker("A"), axis=1)
            data['thread_split_B_{}_{}'.format(*preproc_mode)] = data.apply(get_speaker("B"), axis=1)
            
            def join_speaker(speaker):
                return lambda row: " ".join(row["thread_split_{}_{}_{}".format(speaker, *preproc_mode)])
            
            data["thread_joined_A_{}_{}".format(*preproc_mode)] = data.apply(join_speaker("A"), axis=1)
            data["thread_joined_B_{}_{}".format(*preproc_mode)] = data.apply(join_speaker("B"), axis=1)

            def get_first(speaker):
                return lambda row: " ".join(row["thread_split_{}_{}_{}".format(speaker, *preproc_mode)])
            
            data["start_A_{}_{}".format(*preproc_mode)] = data.apply(get_first("A"), axis=1)
            data["start_B_{}_{}".format(*preproc_mode)] = data.apply(get_first("B"), axis=1)

            tfidf_thread = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), max_features=4000)
            
            data["tfidf_all_{}_{}".format(*preproc_mode)] = tfidf_thread.fit_transform(data["thread_raw"]).toarray().tolist()
            data["tfidf_A_{}_{}".format(*preproc_mode)] = tfidf_thread.transform(data["thread_joined_A_{}_{}".format(*preproc_mode)]).toarray().tolist()
            data["tfidf_B_{}_{}".format(*preproc_mode)] = tfidf_thread.transform(data["thread_joined_B_{}_{}".format(*preproc_mode)]).toarray().tolist()
            data["tfidf_start_A_{}_{}".format(*preproc_mode)] = tfidf_thread.transform(data["start_A_{}_{}".format(*preproc_mode)]).toarray().tolist()
            data["tfidf_start_B_{}_{}".format(*preproc_mode)] = tfidf_thread.transform(data["start_B_{}_{}".format(*preproc_mode)]).toarray().tolist()

            tfidf_context = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), max_features=4000)
            data["tfidf_context_{}_{}".format(*preproc_mode)] = tfidf_context.fit_transform(data["context"]).toarray().tolist()

            count_thread = CountVectorizer(analyzer='word', ngram_range=(1, 2), max_features=4000)
            data["counts_all_{}_{}".format(*preproc_mode)] = count_thread.fit_transform(data["thread_raw"]).toarray().tolist()
            data["counts_A_{}_{}".format(*preproc_mode)] = count_thread.transform(data["thread_joined_A_{}_{}".format(*preproc_mode)]).toarray().tolist()
            data["counts_B_{}_{}".format(*preproc_mode)] = count_thread.transform(data["thread_joined_B_{}_{}".format(*preproc_mode)]).toarray().tolist()
            data["counts_start_A_{}_{}".format(*preproc_mode)] = count_thread.transform(data["start_A_{}_{}".format(*preproc_mode)]).toarray().tolist()
            data["counts_start_B_{}_{}".format(*preproc_mode)] = count_thread.transform(data["start_B_{}_{}".format(*preproc_mode)]).toarray().tolist()

            count_context = CountVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1, 2), max_features=4000)
            data["counts_context_{}_{}".format(*preproc_mode)] = count_context.fit_transform(data["context"]).toarray().tolist()
            
            def run_len(target, func):
                return lambda row: [func((len(list(g)) for person, g in itertools.groupby(row["speaker"]) if person == target), default=0)]

            data["f_max_run_A_{}_{}".format(*preproc_mode)] = data.apply(run_len("A", max), axis=1)
            data["f_max_run_B_{}_{}".format(*preproc_mode)] = data.apply(run_len("B", max), axis=1)
            data["f_min_run_A_{}_{}".format(*preproc_mode)] = data.apply(run_len("A", min), axis=1)
            data["f_min_run_B_{}_{}".format(*preproc_mode)] = data.apply(run_len("B", min), axis=1)

            def typo_count(target):
                return lambda row: [sum(1 for word in preproc(row["thread_joined_{}_{}_{}".format(target, *preproc_mode)]) if word not in words)]

            data["f_typos_A_{}_{}".format(*preproc_mode)] = data.apply(typo_count("A"), axis=1)
            data["f_typos_B_{}_{}".format(*preproc_mode)] = data.apply(typo_count("B"), axis=1)
            data["f_typos_frac_A_{}_{}".format(*preproc_mode)] = data.apply(lambda row: row["f_typos_A_{}_{}".format(*preproc_mode)][0] / (1 + len(preproc(row["thread_joined_A_{}_{}".format(*preproc_mode)]).split())), axis=1)
            data["f_typos_frac_B_{}_{}".format(*preproc_mode)] = data.apply(lambda row: row["f_typos_B_{}_{}".format(*preproc_mode)][0] / (1 + len(preproc(row["thread_joined_B_{}_{}".format(*preproc_mode)]).split())), axis=1)

            
            def relevant_words(target):
                return lambda row: [sum(1 for word in preproc(row["thread_joined_{}_{}_{}".format(target, *preproc_mode)]) if word in preproc(row['context']))]
        
            data["f_relevant_A_{}_{}".format(*preproc_mode)] = data.apply(relevant_words("A"), axis=1)
            data["f_relevant_B_{}_{}".format(*preproc_mode)] = data.apply(relevant_words("B"), axis=1)
            
            print("!")
        return data
            
    if os.path.isdir(path):
        data = pd.concat(
            [
                process_file(os.path.join(path, file))
                for file in os.listdir(path)
                if (
                    file.startswith("train") and train or
                    file.startswith("test") and not train
                ) and file not in exclude
            ]
        )
    else:            
        data = process_file(path)
        
    data = add_features(data)
    
    return data

In [405]:
data = process_data("../data/", train=True)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:38: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


!
!
!
!
!
!


In [406]:
X =((data[['tfidf_all_True_exclude',
 'tfidf_A_True_exclude',
 'tfidf_B_True_exclude',
 'tfidf_start_A_True_exclude',
 'tfidf_start_B_True_exclude',
 'tfidf_context_True_exclude',
 'counts_all_True_exclude',
 'counts_A_True_exclude',
 'counts_B_True_exclude',
 'counts_start_A_True_exclude',
 'counts_start_B_True_exclude',
 'counts_context_True_exclude',
 'f_max_run_A_True_exclude',
 'f_max_run_B_True_exclude',
 'f_min_run_A_True_exclude',
 'f_min_run_B_True_exclude',
 'f_typos_A_True_exclude',
 'f_typos_B_True_exclude',
 'f_relevant_A_True_exclude',
 'f_relevant_B_True_exclude',]]).values)
X = np.stack([np.concatenate(X[i]) for i in range(X.shape[0])])

y = data["qualA"].values

In [395]:
LGBMRegressor(n_estimators=100, num_leaves=1000)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
       fair_c=1.0, gaussian_eta=1.0, huber_delta=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, max_drop=50, min_child_samples=10,
       min_child_weight=5, min_split_gain=0, n_estimators=100, nthread=-1,
       num_leaves=1000, objective='regression', poisson_max_delta_step=0.7,
       reg_alpha=0, reg_lambda=0, seed=0, silent=True, skip_drop=0.5,
       subsample=1, subsample_for_bin=50000, subsample_freq=1,
       uniform_drop=False, xgboost_dart_mode=False)

In [ ]:
cv = cross_val_score(LGBMRegressor(n_estimators=100, num_leaves=1000), X, y, cv=10, verbose=3, scoring=spearmancorr)
cv.mean(), cv.std()

#(0.59851429398955014, 0.060098095791398666)